1. write data do delta lake - managed table
2. write data do delta lake - external table
3. read data from delta lake table
4. read data from delta lake file

blob container demo and mount demo

In [0]:
dbutils.fs.ls("mnt/formula1jf/demo")

Out[1]: [FileInfo(path='dbfs:/mnt/formula1jf/demo/race_results_python/', name='race_results_python/', size=0, modificationTime=1740563009000),
 FileInfo(path='dbfs:/mnt/formula1jf/demo/race_results_sql/', name='race_results_sql/', size=0, modificationTime=1740563059000),
 FileInfo(path='dbfs:/mnt/formula1jf/demo/results_external/', name='results_external/', size=0, modificationTime=1740569782000),
 FileInfo(path='dbfs:/mnt/formula1jf/demo/results_managed/', name='results_managed/', size=0, modificationTime=1740569787000),
 FileInfo(path='dbfs:/mnt/formula1jf/demo/results_partition/', name='results_partition/', size=0, modificationTime=1740570515000)]

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS f1_demo
LOCATION '/mnt/formula1jf/demo'

In [0]:
results_df = spark.read.option("inferSchema", True).json("/mnt/formula1jf/raw/2021-03-28/results.json")

In [0]:
results_df.write.format("delta").mode("overwrite").save("/mnt/formula1jf/demo/results_external")
results_df.write.format("delta").mode("overwrite").saveAsTable("f1_demo.results_managed")

In [0]:
%sql
DROP TABLE f1_demo.results_managed

In [0]:
%sql
CREATE TABLE IF NOT EXISTS f1_demo.results_external
USING DELTA
LOCATION '/mnt/formula1jf/demo/results_external';

In [0]:
%sql
SELECT * FROM f1_demo.results_external;

constructorId,driverId,fastestLap,fastestLapSpeed,fastestLapTime,grid,laps,milliseconds,number,points,position,positionOrder,positionText,raceId,rank,resultId,statusId,time
131,1,44,207.235,1:34.015,2,56,5523897,44,25,1,1,1,1052,4,24966,1,1:32:03.897
9,830,41,208.984,1:33.228,1,56,5524642,33,18,2,2,2,1052,2,24967,1,+0.745
131,822,56,211.566,1:32.090,3,56,5561280,77,16,3,3,3,1052,1,24968,1,+37.383
1,846,38,206.398,1:34.396,7,56,5570363,4,12,4,4,4,1052,6,24969,1,+46.466
9,815,44,207.334,1:33.970,0,56,5575944,11,10,5,5,5,1052,3,24970,1,+52.047
6,844,39,205.112,1:34.988,4,56,5582987,16,8,6,6,6,1052,11,24971,1,+59.090
1,817,36,205.233,1:34.932,6,56,5589901,3,6,7,7,7,1052,10,24972,1,+66.004
6,832,48,206.151,1:34.509,8,56,5590997,55,4,8,8,8,1052,7,24973,1,+67.100
213,852,38,205.603,1:34.761,13,56,5609589,22,2,9,9,9,1052,8,24974,1,+85.692
117,840,31,205.378,1:34.865,10,56,5610610,18,1,10,10,10,1052,9,24975,1,+86.713


In [0]:
results_external_df = spark.read.format("delta").load("mnt/formula1jf/demo/results_external")

In [0]:
display(results_df)

In [0]:
results_df.write.format("delta").mode("overwrite").partitionBy("constructorId").saveAsTable("f1_demo.results_partition")

In [0]:
%sql
SHOW PARTITIONS f1_demo.results_partition;

1. Update delta table
2. Delete from delta table

In [0]:
%sql
SELECT * FROM f1_demo.results_managed;

constructorId,driverId,fastestLap,fastestLapSpeed,fastestLapTime,grid,laps,milliseconds,number,points,position,positionOrder,positionText,raceId,rank,resultId,statusId,time
131,1,44,207.235,1:34.015,2,56,5523897,44,10,1,1,1,1052,4,24966,1,1:32:03.897
9,830,41,208.984,1:33.228,1,56,5524642,33,9,2,2,2,1052,2,24967,1,+0.745
131,822,56,211.566,1:32.090,3,56,5561280,77,8,3,3,3,1052,1,24968,1,+37.383
1,846,38,206.398,1:34.396,7,56,5570363,4,7,4,4,4,1052,6,24969,1,+46.466
9,815,44,207.334,1:33.970,0,56,5575944,11,6,5,5,5,1052,3,24970,1,+52.047
6,844,39,205.112,1:34.988,4,56,5582987,16,5,6,6,6,1052,11,24971,1,+59.090
1,817,36,205.233,1:34.932,6,56,5589901,3,4,7,7,7,1052,10,24972,1,+66.004
6,832,48,206.151,1:34.509,8,56,5590997,55,3,8,8,8,1052,7,24973,1,+67.100
213,852,38,205.603,1:34.761,13,56,5609589,22,2,9,9,9,1052,8,24974,1,+85.692
117,840,31,205.378,1:34.865,10,56,5610610,18,1,10,10,10,1052,9,24975,1,+86.713


In [0]:
%sql
UPDATE f1_demo.results_managed
  SET points = 11 - position
  WHERE position <= 10;

num_affected_rows
10


In [0]:
%sql
SELECT * FROM f1_demo.results_managed

constructorId,driverId,fastestLap,fastestLapSpeed,fastestLapTime,grid,laps,milliseconds,number,points,position,positionOrder,positionText,raceId,rank,resultId,statusId,time
131,1,44,207.235,1:34.015,2,56,5523897,44,20,1,1,1,1052,4,24966,1,1:32:03.897
9,830,41,208.984,1:33.228,1,56,5524642,33,19,2,2,2,1052,2,24967,1,+0.745
131,822,56,211.566,1:32.090,3,56,5561280,77,18,3,3,3,1052,1,24968,1,+37.383
1,846,38,206.398,1:34.396,7,56,5570363,4,17,4,4,4,1052,6,24969,1,+46.466
9,815,44,207.334,1:33.970,0,56,5575944,11,16,5,5,5,1052,3,24970,1,+52.047
6,844,39,205.112,1:34.988,4,56,5582987,16,15,6,6,6,1052,11,24971,1,+59.090
1,817,36,205.233,1:34.932,6,56,5589901,3,14,7,7,7,1052,10,24972,1,+66.004
6,832,48,206.151,1:34.509,8,56,5590997,55,13,8,8,8,1052,7,24973,1,+67.100
213,852,38,205.603,1:34.761,13,56,5609589,22,12,9,9,9,1052,8,24974,1,+85.692
117,840,31,205.378,1:34.865,10,56,5610610,18,11,10,10,10,1052,9,24975,1,+86.713


In [0]:
from delta.tables import DeltaTable
deltaTable = DeltaTable.forPath(spark, "/mnt/formula1jf/demo/results_managed")

deltaTable.update("position <= 10", {"points": "21 - position"} )

In [0]:
%sql
DELETE FROM f1_demo.results_managed
WHERE position > 10;

num_affected_rows
8


In [0]:
deltaTable.delete("points = 0")

In [0]:
%sql
SELECT * FROM f1_demo.results_managed

constructorId,driverId,fastestLap,fastestLapSpeed,fastestLapTime,grid,laps,milliseconds,number,points,position,positionOrder,positionText,raceId,rank,resultId,statusId,time
131,1,44,207.235,1:34.015,2,56,5523897,44,20,1,1,1,1052,4,24966,1,1:32:03.897
9,830,41,208.984,1:33.228,1,56,5524642,33,19,2,2,2,1052,2,24967,1,+0.745
131,822,56,211.566,1:32.090,3,56,5561280,77,18,3,3,3,1052,1,24968,1,+37.383
1,846,38,206.398,1:34.396,7,56,5570363,4,17,4,4,4,1052,6,24969,1,+46.466
9,815,44,207.334,1:33.970,0,56,5575944,11,16,5,5,5,1052,3,24970,1,+52.047
6,844,39,205.112,1:34.988,4,56,5582987,16,15,6,6,6,1052,11,24971,1,+59.090
1,817,36,205.233,1:34.932,6,56,5589901,3,14,7,7,7,1052,10,24972,1,+66.004
6,832,48,206.151,1:34.509,8,56,5590997,55,13,8,8,8,1052,7,24973,1,+67.100
213,852,38,205.603,1:34.761,13,56,5609589,22,12,9,9,9,1052,8,24974,1,+85.692
117,840,31,205.378,1:34.865,10,56,5610610,18,11,10,10,10,1052,9,24975,1,+86.713


Upsert using merge

In [0]:
drivers_day1_df = spark.read \
    .option("inferSchema", True) \
    .json("/mnt/formula1jf/raw/2021-03-28/drivers.json") \
    .filter("driverId <= 10") \
    .select("driverId", "dob", "name.forename", "name.surname")

In [0]:
from pyspark.sql.functions import upper

drivers_day2_df = spark.read \
    .option("inferSchema", True) \
    .json("/mnt/formula1jf/raw/2021-03-28/drivers.json") \
    .filter("driverId BETWEEN 6 AND 15") \
    .select("driverId", "dob", upper("name.forename").alias("forename"), upper("name.surname").alias("surname") )

In [0]:
display(drivers_day2_df)

driverId,dob,forename,surname
6,1985-01-11,KAZUKI,NAKAJIMA
7,1979-02-28,SÉBASTIEN,BOURDAIS
8,1979-10-17,KIMI,RÄIKKÖNEN
9,1984-12-07,ROBERT,KUBICA
10,1982-03-18,TIMO,GLOCK
11,1977-01-28,TAKUMA,SATO
12,1985-07-25,NELSON,PIQUET JR.
13,1981-04-25,FELIPE,MASSA
14,1971-03-27,DAVID,COULTHARD
15,1974-07-13,JARNO,TRULLI


In [0]:
from pyspark.sql.functions import upper

drivers_day3_df = spark.read \
    .option("inferSchema", True) \
    .json("/mnt/formula1jf/raw/2021-03-28/drivers.json") \
    .filter("driverId BETWEEN 1 AND 5 OR driverId BETWEEN 15 AND 20") \
    .select("driverId", "dob", upper("name.forename").alias("forename"), upper("name.surname").alias("surname") )

In [0]:
drivers_day1_df.createOrReplaceTempView("drivers_day1")
drivers_day2_df.createOrReplaceTempView("drivers_day2")

In [0]:
%sql
CREATE TABLE IF NOT EXISTS f1_demo.drivers_merge(
  driverId INT,
  dob DATE,
  forename STRING,
  surname STRING,
  createdDate DATE,
  updatedDATE DATE
)
USING DELTA

In [0]:
%sql
MERGE INTO f1_demo.drivers_merge tgt
USING drivers_day1 upd
ON tgt.driverId = upd.driverId
WHEN MATCHED THEN
  UPDATE SET tgt.dob = upd.dob,
            tgt.forename = upd.forename,
            tgt.surname = upd.surname,
            tgt.updatedDATE = current_timestamp
WHEN NOT MATCHED THEN
  INSERT (driverId, dob, forename, surname, createdDate) VALUES (driverId, dob, forename, surname, current_timestamp)

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
10,0,0,10


In [0]:
%sql
SELECT * FROM f1_demo.drivers_merge

driverId,dob,forename,surname,createdDate,updatedDATE
1,1985-01-07,LEWIS,HAMILTON,2025-02-26,2025-02-26
2,1977-05-10,NICK,HEIDFELD,2025-02-26,2025-02-26
3,1985-06-27,NICO,ROSBERG,2025-02-26,2025-02-26
4,1981-07-29,FERNANDO,ALONSO,2025-02-26,2025-02-26
5,1981-10-19,HEIKKI,KOVALAINEN,2025-02-26,2025-02-26
15,1974-07-13,JARNO,TRULLI,2025-02-26,2025-02-26
16,1983-01-11,ADRIAN,SUTIL,2025-02-26,null
17,1976-08-27,MARK,WEBBER,2025-02-26,null
18,1980-01-19,JENSON,BUTTON,2025-02-26,null
19,1979-04-18,ANTHONY,DAVIDSON,2025-02-26,null


In [0]:
%sql
MERGE INTO f1_demo.drivers_merge tgt
USING drivers_day2 upd
ON tgt.driverId = upd.driverId
WHEN MATCHED THEN
  UPDATE SET tgt.dob = upd.dob,
            tgt.forename = upd.forename,
            tgt.surname = upd.surname,
            tgt.updatedDATE = current_timestamp
WHEN NOT MATCHED THEN
  INSERT (driverId, dob, forename, surname, createdDate) VALUES (driverId, dob, forename, surname, current_timestamp)

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
10,5,0,5


In [0]:
from pyspark.sql.functions import current_timestamp

deltaTable = DeltaTable.forPath(spark, "/mnt/formula1jf/demo/drivers_merge")

deltaTable.alias("tgt").merge(  
    drivers_day3_df.alias("upd"),
    "tgt.driverId = upd.driverId") \
    .whenMatchedUpdate( set = {"dob": "upd.dob", "forename": "upd.forename", "surname": "upd.surname", "updatedDate": "current_timestamp()"}) \
    .whenNotMatchedInsert( values = 
        {
        "driverId": "upd.driverId",
        "dob": "upd.dob",
        "forename": "upd.forename",
        "surname": "upd.surname",
        "createdDate": "current_timestamp()"
        }
    ) \
    .execute()


1. History & Versioning
2. Time Travel
3. Vacuum

In [0]:
%sql
DESC HISTORY f1_demo.drivers_merge;

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
3,2025-02-26T13:57:19.000+0000,3959459284846739,joanaflores@outlook.com,MERGE,"Map(predicate -> [""(cast(driverId#11176 as bigint) = driverId#7862L)""], matchedPredicates -> [{""actionType"":""update""}], notMatchedPredicates -> [{""actionType"":""insert""}])",null,List(2245422955904076),0213-111550-l2zg2q9n,2,WriteSerializable,false,"Map(numTargetRowsCopied -> 9, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 2, executionTimeMs -> 3263, materializeSourceTimeMs -> 363, numTargetRowsInserted -> 5, scanTimeMs -> 1368, numTargetRowsUpdated -> 6, numOutputRows -> 20, numTargetChangeFilesAdded -> 0, numSourceRows -> 11, numTargetFilesRemoved -> 2, rewriteTimeMs -> 1265)",null,Databricks-Runtime/11.3.x-scala2.12
2,2025-02-26T13:50:57.000+0000,3959459284846739,joanaflores@outlook.com,MERGE,"Map(predicate -> [""(cast(driverId#10128 as bigint) = driverId#7793L)""], matchedPredicates -> [{""actionType"":""update""}], notMatchedPredicates -> [{""actionType"":""insert""}])",null,List(2245422955904076),0213-111550-l2zg2q9n,1,WriteSerializable,false,"Map(numTargetRowsCopied -> 5, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 2, executionTimeMs -> 3263, materializeSourceTimeMs -> 291, numTargetRowsInserted -> 5, scanTimeMs -> 1647, numTargetRowsUpdated -> 5, numOutputRows -> 15, numTargetChangeFilesAdded -> 0, numSourceRows -> 10, numTargetFilesRemoved -> 1, rewriteTimeMs -> 1007)",null,Databricks-Runtime/11.3.x-scala2.12
1,2025-02-26T13:48:49.000+0000,3959459284846739,joanaflores@outlook.com,MERGE,"Map(predicate -> [""(cast(driverId#9102 as bigint) = driverId#9051L)""], matchedPredicates -> [{""actionType"":""update""}], notMatchedPredicates -> [{""actionType"":""insert""}])",null,List(2245422955904076),0213-111550-l2zg2q9n,0,WriteSerializable,false,"Map(numTargetRowsCopied -> 0, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 1, executionTimeMs -> 2812, materializeSourceTimeMs -> 453, numTargetRowsInserted -> 10, scanTimeMs -> 1524, numTargetRowsUpdated -> 0, numOutputRows -> 10, numTargetChangeFilesAdded -> 0, numSourceRows -> 10, numTargetFilesRemoved -> 0, rewriteTimeMs -> 719)",null,Databricks-Runtime/11.3.x-scala2.12
0,2025-02-26T13:41:37.000+0000,3959459284846739,joanaflores@outlook.com,CREATE TABLE,"Map(isManaged -> true, description -> null, partitionBy -> [], properties -> {})",null,List(2245422955904076),0213-111550-l2zg2q9n,null,WriteSerializable,true,Map(),null,Databricks-Runtime/11.3.x-scala2.12


In [0]:
%sql
SELECT * FROM f1_demo.drivers_merge
VERSION AS OF 1;

driverId,dob,forename,surname,createdDate,updatedDATE
1,1985-01-07,Lewis,Hamilton,2025-02-26,null
2,1977-05-10,Nick,Heidfeld,2025-02-26,null
3,1985-06-27,Nico,Rosberg,2025-02-26,null
4,1981-07-29,Fernando,Alonso,2025-02-26,null
5,1981-10-19,Heikki,Kovalainen,2025-02-26,null
6,1985-01-11,Kazuki,Nakajima,2025-02-26,null
7,1979-02-28,Sébastien,Bourdais,2025-02-26,null
8,1979-10-17,Kimi,Räikkönen,2025-02-26,null
9,1984-12-07,Robert,Kubica,2025-02-26,null
10,1982-03-18,Timo,Glock,2025-02-26,null


In [0]:
%sql
SELECT * FROM f1_demo.drivers_merge
TIMESTAMP AS OF '2025-02-26T13:48:49.000+00:00';

driverId,dob,forename,surname,createdDate,updatedDATE
1,1985-01-07,Lewis,Hamilton,2025-02-26,null
2,1977-05-10,Nick,Heidfeld,2025-02-26,null
3,1985-06-27,Nico,Rosberg,2025-02-26,null
4,1981-07-29,Fernando,Alonso,2025-02-26,null
5,1981-10-19,Heikki,Kovalainen,2025-02-26,null
6,1985-01-11,Kazuki,Nakajima,2025-02-26,null
7,1979-02-28,Sébastien,Bourdais,2025-02-26,null
8,1979-10-17,Kimi,Räikkönen,2025-02-26,null
9,1984-12-07,Robert,Kubica,2025-02-26,null
10,1982-03-18,Timo,Glock,2025-02-26,null


In [0]:
df = spark.read.format("delta").option("timestampAsOf", '2025-02-26T13:48:49.000+00:00')
display(df)

In [0]:
%sql
VACUUM f1_demo.drivers_merge

path
dbfs:/mnt/formula1jf/demo/drivers_merge


In [0]:
%sql
SET spark.databricks.delta.retentionDurationCheck.enabled = false; --because retention hours < 168
VACUUM f1_demo.drivers_merge --removes history and makes impossible to access previous versions, but the stay on versioning history
RETAIN 0 HOURS

path
dbfs:/mnt/formula1jf/demo/drivers_merge


In [0]:
%sql
DELETE FROM f1_demo.drivers_merge WHERE driverId = 1;

num_affected_rows
1


In [0]:
%sql
MERGE INTO f1_demo.drivers_merge tgt
USING f1_demo.drivers_merge VERSION AS OF 3 src
  ON tgt.driverId = src.driverId
  WHEN NOT MATCHED THEN
    INSERT *

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
1,0,0,1


In [0]:
%sql
DESC HISTORY f1_demo.drivers_merge 

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
9,2025-02-26T14:21:54.000+0000,3959459284846739,joanaflores@outlook.com,MERGE,"Map(predicate -> [""(driverId#15924 = driverId#15930)""], matchedPredicates -> [], notMatchedPredicates -> [{""actionType"":""insert""}])",null,List(2245422955904076),0213-111550-l2zg2q9n,8,WriteSerializable,false,"Map(numTargetRowsCopied -> 0, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 1, executionTimeMs -> 1695, materializeSourceTimeMs -> 5, numTargetRowsInserted -> 1, scanTimeMs -> 0, numTargetRowsUpdated -> 0, numOutputRows -> 1, numTargetChangeFilesAdded -> 0, numSourceRows -> 25, numTargetFilesRemoved -> 0, rewriteTimeMs -> 1651)",null,Databricks-Runtime/11.3.x-scala2.12
8,2025-02-26T14:19:32.000+0000,3959459284846739,joanaflores@outlook.com,DELETE,"Map(predicate -> [""(driverId#15332 = 1)""])",null,List(2245422955904076),0213-111550-l2zg2q9n,7,WriteSerializable,false,"Map(numRemovedFiles -> 1, numCopiedRows -> 10, numAddedChangeFiles -> 0, executionTimeMs -> 1444, numDeletedRows -> 1, scanTimeMs -> 825, numAddedFiles -> 1, rewriteTimeMs -> 615)",null,Databricks-Runtime/11.3.x-scala2.12
7,2025-02-26T14:17:31.000+0000,3959459284846739,joanaflores@outlook.com,VACUUM END,Map(status -> COMPLETED),null,List(2245422955904076),0213-111550-l2zg2q9n,6,SnapshotIsolation,true,"Map(numDeletedFiles -> 3, numVacuumedDirectories -> 1)",null,Databricks-Runtime/11.3.x-scala2.12
6,2025-02-26T14:17:26.000+0000,3959459284846739,joanaflores@outlook.com,VACUUM START,"Map(retentionCheckEnabled -> false, specifiedRetentionMillis -> 0, defaultRetentionMillis -> 604800000)",null,List(2245422955904076),0213-111550-l2zg2q9n,5,SnapshotIsolation,true,"Map(numFilesToDelete -> 3, sizeOfDataToDelete -> 6125)",null,Databricks-Runtime/11.3.x-scala2.12
5,2025-02-26T14:14:13.000+0000,3959459284846739,joanaflores@outlook.com,VACUUM END,Map(status -> COMPLETED),null,List(2245422955904076),0213-111550-l2zg2q9n,4,SnapshotIsolation,true,"Map(numDeletedFiles -> 0, numVacuumedDirectories -> 1)",null,Databricks-Runtime/11.3.x-scala2.12
4,2025-02-26T14:14:08.000+0000,3959459284846739,joanaflores@outlook.com,VACUUM START,"Map(retentionCheckEnabled -> true, defaultRetentionMillis -> 604800000)",null,List(2245422955904076),0213-111550-l2zg2q9n,3,SnapshotIsolation,true,"Map(numFilesToDelete -> 0, sizeOfDataToDelete -> 0)",null,Databricks-Runtime/11.3.x-scala2.12
3,2025-02-26T13:57:19.000+0000,3959459284846739,joanaflores@outlook.com,MERGE,"Map(predicate -> [""(cast(driverId#11176 as bigint) = driverId#7862L)""], matchedPredicates -> [{""actionType"":""update""}], notMatchedPredicates -> [{""actionType"":""insert""}])",null,List(2245422955904076),0213-111550-l2zg2q9n,2,WriteSerializable,false,"Map(numTargetRowsCopied -> 9, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 2, executionTimeMs -> 3263, materializeSourceTimeMs -> 363, numTargetRowsInserted -> 5, scanTimeMs -> 1368, numTargetRowsUpdated -> 6, numOutputRows -> 20, numTargetChangeFilesAdded -> 0, numSourceRows -> 11, numTargetFilesRemoved -> 2, rewriteTimeMs -> 1265)",null,Databricks-Runtime/11.3.x-scala2.12
2,2025-02-26T13:50:57.000+0000,3959459284846739,joanaflores@outlook.com,MERGE,"Map(predicate -> [""(cast(driverId#10128 as bigint) = driverId#7793L)""], matchedPredicates -> [{""actionType"":""update""}], notMatchedPredicates -> [{""actionType"":""insert""}])",null,List(2245422955904076),0213-111550-l2zg2q9n,1,WriteSerializable,false,"Map(numTargetRowsCopied -> 5, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 2, executionTimeMs -> 3263, materializeSourceTimeMs -> 291, numTargetRowsInserted -> 5, scanTimeMs -> 1647, numTargetRowsUpdated -> 5, numOutputRows -> 15, numTargetChangeFilesAdded -> 0, numSourceRows -> 10, numTargetFilesRemoved -> 1, rewriteTimeMs -> 1007)",null,Databricks-Runtime/11.3.x-scala2.12
1,2025-02-26T13:48:49.000+0000,3959459284846739,

### Transaction Logs

In [0]:
%sql
CREATE TABLE IF NOT EXISTS f1_demo.drivers_txn(
  driverId INT,
  dob DATE,
  forename STRING,
  surname STRING,
  createdDate DATE,
  updatedDate DATE
)
USING DELTA

In [0]:
%sql
INSERT INTO f1_demo.drivers_txn
SELECT * FROM f1_demo.drivers_merge
  WHERE driverId = 1;

num_affected_rows,num_inserted_rows
1,1


In [0]:
%sql
DESC HISTORY f1_demo.drivers_txn;

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
1,2025-02-26T14:29:10.000+0000,3959459284846739,joanaflores@outlook.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(2245422955904076),0213-111550-l2zg2q9n,0,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 1836)",null,Databricks-Runtime/11.3.x-scala2.12
0,2025-02-26T14:26:49.000+0000,3959459284846739,joanaflores@outlook.com,CREATE TABLE,"Map(isManaged -> true, description -> null, partitionBy -> [], properties -> {})",null,List(2245422955904076),0213-111550-l2zg2q9n,null,WriteSerializable,true,Map(),null,Databricks-Runtime/11.3.x-scala2.12


In [0]:
%sql
DELETE FROM f1_demo.drivers_txn
WHERE driverId = 1;

num_affected_rows
1


### Convert parquet to delta

In [0]:
%sql
CREATE TABLE IF NOT EXISTS f1_demo.convert_to_delta(
  driverId INT,
  dob DATE,
  forename STRING,
  surname STRING,
  createdDate DATE,
  updatedDate DATE
)
USING PARQUET

In [0]:
%sql
INSERT INTO f1_demo.convert_to_delta
  SELECT * FROM f1_demo.drivers_merge

In [0]:
%sql
CONVERT TO DELTA f1_demo.convert_to_delta;

In [0]:
df = spark.table("f1_demo.convert_to_delta")
df.write.format("parquet").save("/mnt/formula1jf/demo/convert_to_delta_new")

In [0]:
%sql
CONVERT TO DELTA parquet.`/mnt/formula1jf/demo/convert_to_delta_new`